In [33]:
import ebaysdk
import datetime
import sys
import plotly as py
import plotly.graph_objs as go
import pandas as pd

from ebaysdk.finding import Connection as finding
from plotly.graph_objs import Scatter, Figure, Layout

py.offline.init_notebook_mode(connected=True)

In [34]:
# Constants and initialization

exclusions = "-HP -SP -error -psa -PL -LP -set -collection -flash -online -digital -playset -(4) -(2) -(3) -3x -x3 -2x -x2 -4x -x4 -lot -bundle -prerelease -sealed -china -fake"
categoryId = '2611'

api = finding(siteid='EBAY-US', appid='ClaudeAm-PokemonT-PRD-85d705b3d-fceb8191', config_file=None)


In [35]:
# EBay API call
# Returns dictionary of completed items which is then converted to Pandas dataframe

def findCompletedItems(searchstring):
    api.execute('findCompletedItems', {
        'keywords': [searchstring],
        'categoryId' : categoryId,
        'itemFilter': [
            {'name': 'Currency', 'value': 'USD'},
            {'name': 'ListedIn', 'value': 'EBAY-US'},
            {'name': 'LocatedIn', 'value': 'US'},
            {'name': 'SoldItemsOnly', 'value': 'true'}
        ],
        'paginationInput': {
            'entriesPerPage': '200',
            'pageNumber': '1'
        },
        'sortOrder': 'EndTimeSoonest'
    })

    dict_items = api.response.dict()
    list_results = []
    
    #DEBUG
    #for k, v in dict_items.iteritems():
    #    print k, v

        
    for item in dict_items['searchResult']['item']:
        # print "ItemID: %s" % item['itemId']
        # print "Title: %s" % item['title']
        if item['shippingInfo']['shippingType'] != 'Calculated' and item['shippingInfo']['shippingType'] != 'FreePickup':

            price = float(item['sellingStatus']['convertedCurrentPrice']['value'])
            shipping = float(item['shippingInfo']['shippingServiceCost']['value'])
            totalcost = price + shipping
            saledate = item['listingInfo']['endTime'][:item['listingInfo']['endTime'].find('T')]
            saledatetime = item['listingInfo']['endTime']

            dt_saledatetime = datetime.datetime.strptime(saledatetime, '%Y-%m-%dT%H:%M:%S.000Z')

            # print str.format(saledate) + ' ' + str(totalcost)
            list_results.append((dt_saledatetime, totalcost)) 
            
    
    df_results = pd.DataFrame(list_results, columns=['Date', 'Price'])
    
    return df_results
        

In [36]:
def createChart(df_input,keywords):
    
    df_items_adj = df_input.sort_values(by='Date').set_index('Date')
    df_items_sma = df_items_adj.rolling(6).mean()
    
    layout = dict(title = keywords + ' Price History',
              showlegend = False,
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'All-In Prince'),
              )

    line0 = go.Scatter(
              x=df_items_adj.index,
              y=df_items_adj.Price,
              mode = 'lines+markers',
              name = searchstring + ' Price History',
              marker = dict(symbol = 'diamond'),
              line = dict(color = 'green', width=1)
            )

    line1 = go.Scatter(
              x=df_items_sma.index,
              y=df_items_sma.Price,
              mode = 'lines',
              name = searchstring + ' Price History Avg',
              line = dict(color = 'blue', width=4)
            )

    data = [line0, line1]

    fig = dict(data=data, layout=layout)
    py.offline.iplot(fig)
    

In [45]:
list_search = [
    'Ancient Origins Groudon 97/98'
]

for list_item in list_search:
    searchstring = list_item + ' ' + exclusions
    df_items = findCompletedItems(searchstring)
    createChart(df_items,list_item)